In [38]:
import requests
import base64
import datetime
from urllib.parse import urlencode

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
import time
import re
import json

import datetime

In [3]:
dfGenres = pd.read_csv('dfGenres.csv')
totalHist = pd.read_csv('totalHist.csv')

In [4]:
dfGenres

,artistName,genres
0,Imagine Dragons,"['modern rock', 'rock']"
1,iLOVEFRiDAY,['viral pop']
2,Panic! At The Disco,"['baroque pop', 'emo', 'modern rock', 'pop punk']"
3,Oliver Tree,['alternative hip hop']
4,88rising,"['asian american hip hop', 'pop']"
...,...,...
1536,Anuel AA,"['latin', 'reggaeton flow', 'trap latino']"
1537,Enrique Iglesias,"['dance pop', 'latin', 'latin pop', 'mexican p..."
1538,Myke Towers,['trap latino']
1539,Pedro CapÃ³,"['latin', 'latin pop', 'puerto rican pop']"


In [5]:
def convToList(row):
    if pd.isna(row['genres']):
        return['No Genre']
    else:
        return row['genres'][1:-1].replace('\'', '').split(', ')


In [6]:
dfGenres['genres'] = dfGenres.apply(convToList, axis=1)

In [7]:
genreList = []
for genres in dfGenres['genres']:
    for genre in genres:
        genreList.append(genre)

In [8]:
uniqueGenres = pd.Series(genreList).unique()

In [9]:
for genre in uniqueGenres:
    dfGenres[genre] = 0
for genre in uniqueGenres:
    totalHist[genre] = 0

In [10]:
def fillOutGenres(row):    
    for genre in row['genres']:
        row[genre] = 1
    return row
    

In [11]:
dfGenres = dfGenres.apply(fillOutGenres, axis=1)

In [12]:
dfGenres

,artistName,genres,modern rock,rock,viral pop,baroque pop,emo,pop punk,alternative hip hop,asian american hip hop,...,polynesian pop,experimental,experimental rock,popgaze,noise punk,deconstructed club,modern salsa,salsa,reggaeton flow,puerto rican pop
0,Imagine Dragons,"[modern rock, rock]",1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,iLOVEFRiDAY,[viral pop],0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Panic! At The Disco,"[baroque pop, emo, modern rock, pop punk]",1,0,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Oliver Tree,[alternative hip hop],0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,88rising,"[asian american hip hop, pop]",0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1536,Anuel AA,"[latin, reggaeton flow, trap latino]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1537,Enrique Iglesias,"[dance pop, latin, latin pop, mexican pop, pop...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1538,Myke Towers,[trap latino],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1539,Pedro CapÃ³,"[latin, latin pop, puerto rican pop]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [13]:
totalHist

,endTime,artistName,trackName,msPlayed,modern rock,rock,viral pop,baroque pop,emo,pop punk,...,polynesian pop,experimental,experimental rock,popgaze,noise punk,deconstructed club,modern salsa,salsa,reggaeton flow,puerto rican pop
0,2018-11-14 00:01,Imagine Dragons,Stuck,190853,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2018-11-14 00:04,Imagine Dragons,Love,166360,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2018-11-14 00:08,Imagine Dragons,Birds,219440,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2018-11-14 00:12,Imagine Dragons,Burn Out,273880,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2018-11-14 00:16,Imagine Dragons,Real Life,247546,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30822,2019-11-14 23:32,My Chemical Romance,Welcome to the Black Parade,311106,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
30823,2019-11-14 23:34,Tony Molina,Jasper's Theme,121509,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
30824,2019-11-14 23:38,Imagine Dragons,Cool Out,217866,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
30825,2019-11-14 23:41,5 Seconds of Summer,If Walls Could Talk,182292,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [60]:
def convToDatetime(timeStr):
    return datetime.datetime.strptime(timeStr, '%Y-%m-%d %H:%M')

def getYear(x):
    return x.year

def getMonth(x):
    return x.month

def getDay(x):
    return x.day

In [33]:
totalHist['endTime'] = totalHist['endTime'].apply(convToDatetime)

In [62]:
totalHist["year"] = totalHist["endTime"].apply(getYear)
totalHist["month"] = totalHist["endTime"].apply(getMonth)
totalHist["day"] = totalHist["endTime"].apply(getDay)

# 2019 Analysis

In [66]:
df = totalHist[totalHist['year'] == 2019]
df.head()

,endTime,artistName,trackName,msPlayed,modern rock,rock,viral pop,baroque pop,emo,pop punk,...,popgaze,noise punk,deconstructed club,modern salsa,salsa,reggaeton flow,puerto rican pop,year,month,day
3561,2019-01-01 00:03:00,Drake,Now & Forever,169196,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2019,1,1
3562,2019-01-01 00:08:00,Drake,Know Yourself,288489,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2019,1,1
3563,2019-01-01 00:14:00,Drake,No Tellin',310933,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2019,1,1
3564,2019-01-01 00:17:00,Drake,Madonna,177840,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2019,1,1
3565,2019-01-01 00:18:00,Drake,No Tellin',1258,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2019,1,1


In [73]:
monthArtist = pd.pivot_table(df, values='msPlayed', index='artistName', columns='month', aggfunc=np.sum)
monthArtist.head()

month,1,2,3,4,5,6,7,8,9,10,11
artistName,,,,,,,,,,,
$NOT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15350.0,NaN
$uicideBoy$,NaN,267409.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
.Fuelh,NaN,NaN,NaN,NaN,84023.0,NaN,NaN,NaN,NaN,NaN,NaN
.Sinh,197160.0,197160.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,72916.0,72916.0
.anxious.,NaN,441000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [74]:
monthArtist.nlargest(5, 1).index

Index(['Drake', 'Fall Out Boy', 'Kendrick Lamar', 'The Beatles', 'Incubus'], dtype='object', name='artistName')

In [77]:
topArtists = []
for col in monthArtist:
    artists = monthArtist.nlargest(5, col)[col]
    topArtists.append(artists)

In [78]:
topArtists

[artistName
 Drake             44711781.0
 Fall Out Boy      39504850.0
 Kendrick Lamar    25839954.0
 The Beatles       14670065.0
 Incubus           13245363.0
 Name: 1, dtype: float64,
 artistName
 BROCKHAMPTON      70111254.0
 Kendrick Lamar    20454583.0
 Zedd              18774831.0
 Joji              17457017.0
 Eden Project       9848693.0
 Name: 2, dtype: float64,
 artistName
 BROCKHAMPTON           88746364.0
 My Chemical Romance    48320532.0
 Fall Out Boy           31580728.0
 Ocean Wisdom           27015151.0
 frederic               26737546.0
 Name: 3, dtype: float64,
 artistName
 tricot          24371346.0
 Joji            21535911.0
 Roy Blair       18296642.0
 Ocean Wisdom    18084812.0
 BROCKHAMPTON    15659716.0
 Name: 4, dtype: float64,
 artistName
 PUP                   56607800.0
 Tyler, The Creator    35335020.0
 tricot                30825080.0
 Last Dinosaurs        28563486.0
 Rob Thomas            20934558.0
 Name: 5, dtype: float64,
 artistName
 Injury Reser